<a href="https://colab.research.google.com/github/daar1518/Capstone-Project/blob/main/capstone-ml/Processed_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from operator import add
import seaborn as sns

%matplotlib inline

data = pd.read_csv('heart_2020_cleaned.csv')

# Display the first few rows of the dataset
print(data.head())

# Check the data types of each column
print(data.dtypes)

# Check for missing values
print(data.isnull().sum())

# Explore summary statistics
print(data.describe())


FileNotFoundError: ignored

In [ ]:
data.drop(['Race', ],axis=1,inplace=True)
data.head()

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data['AgeCategory']=data['AgeCategory'].replace(['18-24','25-29'],'0') #young
data['AgeCategory']=data['AgeCategory'].replace(['30-34','35-39','40-44','45-49'],'1') #adult
data['AgeCategory']=data['AgeCategory'].replace(['50-54','55-59','60-64','65-69'],'2') #old
data['AgeCategory']=data['AgeCategory'].replace(['70-74','75-79','80 or older'],'3') #elder
data.head()

In [ ]:
sns.countplot(x='HeartDisease',data=data)
plt.show()
result = data.HeartDisease.value_counts()
print(f"There are {result[0]} patients without heart disease and {result[1]} patients with the disease")

In [ ]:
#female = 0, male = 1
#diabetic: yes=2, risk =1, no= 1
#genhealth: poor=0, fair=1, good = 2, excellent = 3, very good =4

from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()

for col in data:
    if col != 'BMI':
      data[col]=label.fit_transform(data[col])
data.head()


In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(data.corr(), annot = True)
plt.show()

In [ ]:
pip install boruta

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [ ]:
#define the features
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

forest = RandomForestClassifier(n_estimators=1000, n_jobs=-1, class_weight='balanced')

# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2)

# find all relevant features
feat_selector.fit(X, y)

In [ ]:
# show the most important features
important_feat = data.columns[:-1][feat_selector.support_].tolist()
important_feat

In [ ]:
# select the top 9 features
top_feat = data.columns[:-1][feat_selector.ranking_ <9].tolist()
top_feat

In [ ]:
import statsmodels.api as sm

X_top = data[top_feat]
y = data['HeartDisease']

res = sm.Logit(y,X_top).fit()
res.summary()

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

X = data[top_feat]
y = data.iloc[:,-1]

# the numbers before smote
num_before = dict(Counter(y))

#perform smoting

# define pipeline
over = SMOTE(sampling_strategy=0.8)
under = RandomUnderSampler(sampling_strategy=0.8)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# transform the dataset
X_smote, y_smote = pipeline.fit_resample(X, y)

#the numbers after smote
num_after =dict(Counter(y_smote))

print(num_before, num_after)

In [ ]:
from google.colab import files
new_data = pd.concat([pd.DataFrame(X_smote), pd.DataFrame(y_smote)], axis=1)
new_data.columns = ['BMI', 'Smoking', 'PhysicalHealth', 'MentalHealth', 'Sex', 'AgeCategory', 'Diabetic', 'GenHealth', 'SleepTime','HeartDisease']
new_data.to_csv('new_data.csv', index=False)
files.download('new_data.csv')

In [ ]:
new_data.head()

In [ ]:
sns.countplot(x='HeartDisease',data=new_data)
plt.show()
result = new_data.HeartDisease.value_counts()
print(f"There are {result[0]} patients without heart disease and {result[1]} patients with the disease")